In [1]:
import pyLDAvis.gensim
import spacy
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from spacy.tokens import DocBin
import spacy
from tqdm import tqdm
import warnings


warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)


/opt/conda/lib/python3.7/site-packages/past/types/oldstr.py:36: DeprecationWarning: invalid escape sequence \d
  """
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:585: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:627: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:637: D

# Analysis
> 1. Load data and remove duplicated
> 2. LDA (Latent Dirichlet Allocation) to analysis subtopics within the data

In [2]:
#data module
import pandas as pd
from spacy.cli import download


def download_spacy_data():
    download("en_core_web_sm")

    
def validate_data(df):
    # check is as expected and described
    for category in df.category.unique():
        assert category in ['business', 'entertainment', 'politics', 'sport', 'tech']
    assert not any(df.duplicated())


def remove_duplicates(df):
    if any(df.duplicated()):
        dup = df.duplicated().value_counts()
        df = df[~df.duplicated()]
        print(f'{dup[True]} duplicated articles removed from dataset')
    return df

In [3]:
# read the data
df = pd.read_csv('../input/newsgroup20bbcnews/bbc-text.csv')
df = remove_duplicates(df)
validate_data(df)

99 duplicated articles removed from dataset


In [4]:
# summarise
df.describe()

,category,text
count,2126,2126
unique,5,2126
top,sport,tv future in the hands of viewers with home th...
freq,504,1


In [5]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("merge_noun_chunks")

# custom stop words
add_stop_words = ['say', 's', 'mr', 'Mr', 'said', 'says', 'saying', 'today', 'be', 'I']
for stopword in add_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

articles, article = [], []
doc_bin = DocBin()
dictionary = Dictionary()
corpus = []

# clean data
print('Processing articles using spaCy, removing punctuation, stop words, numbers and lemminizing')
for doc in tqdm(nlp.pipe(df.text.values, disable=["tok2vec"])):
    doc_bin.add(doc)
    article = [
        token.lemma_ for token in doc 
        if not token.is_stop 
        and not token.is_punct 
        and not token.like_num
        and not token.is_space]
    dictionary.add_documents([article])
    corpus.append(dictionary.doc2bow(article))

print('Building LDA model')
lda_model = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
lda_model.show_topics()

Processing articles using spaCy, removing punctuation, stop words, numbers and lemminizing


2126it [01:46, 20.01it/s]


Building LDA model


[(0,
  '0.007*"p2p" + 0.007*"music" + 0.006*"uwb" + 0.005*"year" + 0.005*"uk" + 0.005*"new" + 0.004*"sales" + 0.003*"prices" + 0.003*"growth" + 0.003*"networks"'),
 (1,
  '0.006*"people" + 0.006*"mobile" + 0.006*"tv" + 0.005*"technology" + 0.004*"year" + 0.004*"high" + 0.004*"new" + 0.004*"digital" + 0.004*"like" + 0.004*"world"'),
 (2,
  '0.008*"people" + 0.005*"$" + 0.004*"information" + 0.004*"t" + 0.004*"year" + 0.004*"£" + 0.003*"new" + 0.003*"world" + 0.003*"children" + 0.003*"way"'),
 (3,
  '0.010*"m" + 0.010*"$" + 0.007*"sales" + 0.007*"year" + 0.006*"new" + 0.005*"tv" + 0.004*"£" + 0.004*"uk" + 0.004*"company" + 0.003*"people"'),
 (4,
  '0.006*"year" + 0.006*"$" + 0.004*"bank" + 0.004*"new" + 0.003*"m" + 0.003*"dollar" + 0.003*"china" + 0.003*"£" + 0.003*"t" + 0.003*"time"'),
 (5,
  '0.006*"dallaglio" + 0.006*"games" + 0.005*"game" + 0.005*"year" + 0.005*"england" + 0.004*"juninho" + 0.004*"man" + 0.004*"uk" + 0.003*"t" + 0.003*"new"'),
 (6,
  '0.008*"o" + 0.008*"wales" + 0.00

In [6]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.009226  0.009225       1        1  15.327317
1      0.067862  0.011659       2        1  14.694118
3      0.025764  0.047977       3        1  12.865064
9      0.041232 -0.129344       4        1  12.016793
2      0.055167 -0.040853       5        1  11.510276
6     -0.168412 -0.039675       6        1   8.942632
5     -0.067312  0.044253       7        1   6.741844
8      0.005294  0.022001       8        1   6.077903
0      0.023928  0.054319       9        1   6.024612
7      0.025704  0.020439      10        1   5.799442, topic_info=            Term        Freq        Total Category  logprob  loglift
11124  dallaglio  331.000000   331.000000  Default  30.0000  30.0000
17112        p2p  250.000000   250.000000  Default  29.0000  29.0000
2030           o  478.000000   478.000000  Default  28.0000  28.0000
651        sales  702.000000   702.000000  Default  27.0000  27.0000
374      england  680.000000   680.000000  Default  26.0000  26.0000
...          ...         ...          ...      ...      ...      ...
143          new   75.262249  1769.067993  Topic10  -5.7671  -0.3098
419        think   53.745437   533.426257  Topic10  -6.1038   0.5523
822            t   59.715807  1149.929288  Topic10  -5.9984  -0.1105
997        world   56.379876  1102.039955  Topic10  -6.0559  -0.1254
236          way   49.651297   767.011795  Topic10  -6.1830   0.1099

[821 rows x 6 columns], token_table=      Topic      Freq Term
term                      
244       1  0.239298    $
244       2  0.087987    $
244       3  0.353282    $
244       4  0.051992    $
244       5  0.149311    $
...     ...       ...  ...
353       6  0.046465    £
353       7  0.033613    £
353       8  0.045476    £
353       9  0.041522    £
353      10  0.030647    £

[3577 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 4, 10, 3, 7, 6, 9, 1, 8])